# Imports

In [2]:
import os
import sys
import argparse
import numpy as np
import random
import pickle
import time

import torch
import torch.nn as nn
from torch.nn import functional as F
from torch import optim
from torch.utils.data import DataLoader

from utils import getModelName, getNumberOfSpeakers
from data import Dataset
from model import SpeakerClassifier

# Functions

# 1 - Input params

execute and copy

In [3]:
! python args_input_simulation.py

{'train_data_dir': '/scratch/speaker_databases/', 'valid_data_dir': '/scratch/speaker_databases/VoxCeleb-1/wav', 'train_labels_path': 'labels/Vox2.ndx', 'data_mode': 'normal', 'valid_clients': 'labels/clients.ndx', 'valid_impostors': 'labels/impostors.ndx', 'out_dir': './models/model1', 'model_name': 'CNN', 'front_end': 'VGG4L', 'window_size': 3.5, 'randomSlicing': False, 'normalization': 'cmn', 'kernel_size': 1024, 'embedding_size': 400, 'heads_number': 32, 'pooling_method': 'DoubleMHA', 'mask_prob': 0.3, 'scalingFactor': 30.0, 'marginFactor': 0.4, 'annealing': False, 'optimizer': 'Adam', 'learning_rate': 0.0001, 'weight_decay': 0.001, 'batch_size': 64, 'gradientAccumulation': 2, 'max_epochs': 1000000, 'early_stopping': 25, 'print_every': 1000, 'requeue': False, 'validate_every': 10000, 'num_workers': 2}


paste

In [4]:
params_dict = {'train_data_dir': '/scratch/speaker_databases/', 'valid_data_dir': '/scratch/speaker_databases/VoxCeleb-1/wav', 'train_labels_path': 'labels/Vox2.ndx', 'data_mode': 'normal', 'valid_clients': 'labels/clients.ndx', 'valid_impostors': 'labels/impostors.ndx', 'out_dir': './models/model1', 'model_name': 'CNN', 'front_end': 'VGG4L', 'window_size': 3.5, 'randomSlicing': False, 'normalization': 'cmn', 'kernel_size': 1024, 'embedding_size': 400, 'heads_number': 32, 'pooling_method': 'DoubleMHA', 'mask_prob': 0.3, 'scalingFactor': 30.0, 'marginFactor': 0.4, 'annealing': False, 'optimizer': 'Adam', 'learning_rate': 0.0001, 'weight_decay': 0.001, 'batch_size': 64, 'gradientAccumulation': 2, 'max_epochs': 1000000, 'early_stopping': 25, 'print_every': 1000, 'requeue': False, 'validate_every': 10000, 'num_workers': 2}

In [5]:
params = argparse.Namespace(**params_dict)

params.model_name = getModelName(params)
params.num_spkrs = getNumberOfSpeakers(params.train_labels_path) 

print(f"{params.num_spkrs} Speaker Labels")

if not os.path.exists(params.out_dir):
    os.makedirs(params.out_dir)
print(f"Output folder created at {params.out_dir}")
    
with open(params.out_dir + '/' + params.model_name + '_config.pkl', 'wb') as handle:
    pickle.dump(params, handle, protocol=pickle.HIGHEST_PROTOCOL)
print(f"{params.out_dir + '/' + params.model_name + '_config.pkl'}")

5994 Speaker Labels
Output folder created at ./models/model1
./models/model1/CNN_VGG4L_3.5_128batchSize_0.0001lr_0.001weightDecay_1024kernel_400embSize_30.0s_0.4m_DoubleMHA_32_config.pkl


# 2 - Trainer class

In [16]:
class Trainer:

    def __init__(self, params, device):

        self.params = params
        self.device = device
        self.__load_network()
        self.__load_data()
        self.__load_optimizer()
        self.__load_criterion()
        self.__initialize_training_variables()
        
        
    def __load_data(self):
        print('Loading Data and Labels')
        with open(self.params.train_labels_path, 'r') as data_labels_file:
            train_labels = data_labels_file.readlines()

        data_loader_parameters = {
            'batch_size': self.params.batch_size, 
            'shuffle': True, 
            'num_workers': self.params.num_workers,
        }
        
        self.training_generator = DataLoader(Dataset(train_labels, self.params), **data_loader_parameters)
        
        
    def __load_network(self):

        self.net = SpeakerClassifier(self.params, self.device)
        self.net.to(self.device)

        if torch.cuda.device_count() > 1:
            print("Let's use", torch.cuda.device_count(), "GPUs!")
            self.net = nn.DataParallel(self.net)
        
    
    def __load_optimizer(self):
        if self.params.optimizer == 'Adam':
            self.optimizer = optim.Adam(
                self.net.parameters(), 
                lr=self.params.learning_rate, 
                weight_decay=self.params.weight_decay,
            )
        if self.params.optimizer == 'SGD':
            self.optimizer = optim.SGD(
                self.net.parameters(), 
                lr=self.params.learning_rate, 
                weight_decay=self.params.weight_decay,
            )
        if self.params.optimizer == 'RMSprop':
            self.optimizer = optim.RMSprop(
                self.net.parameters(), 
                lr=self.params.learning_rate, 
                weight_decay=self.params.weight_decay,
            )

    
    def __load_criterion(self):
        self.criterion = nn.CrossEntropyLoss()
            
    
    def __initialize_training_variables(self):

        if self.params.requeue:
            self.__load_previous_states()
        else:
            self.step = 0
            self.starting_epoch = 0

        self.best_EER = 50.0
        self.stopping = 0.0

In [17]:
torch.manual_seed(1234)
np.random.seed(1234)
    
#print('Defining Device...')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#print(f"device: {device}")
#print(torch.cuda.get_device_name(0))

trainer = Trainer(params, device)

Loading Data and Labels
